# Day 3 - line segment intersections

- https://adventofcode.com/2019/day/3

This is another [computational geometry](https://en.wikipedia.org/wiki/Computational_geometry) problem; find all the line segment intersections between two sets of lines!

I could go and break open a full [Bently-Ottman algorithm](https://en.wikipedia.org/wiki/Bentley%E2%80%93Ottmann_algorithm) implementation here, but that'd be overkill. First of all, we only need to find where two _distinct_ sets of lines need to intersect, and second, all segments are either strictly horizontal, or vertical. The intersections are only going to be found where a horizontal segment crosses a vertical segment.

## Approach

So my approach is to take all the segments of one of the two lines, and see where they cross the horizontal segments of the other if vertical, and the vertical if they are themselves horizontal. We could use fancy binary trees or bisection to optimize finding the matching segments in the series, but I think grouping the segments by `x` (horizontal) or `y` (vertical) coordinate is plenty.

Then all we have to do is:

- classify segments as horizontal or vertical, and record their fixed value and the extent (range) of their start and stop values in the other direction.
- iterate over the segments of one line, A
  - loop over the `extent` values from start to end, as `pos`
    - find all segments on the other line, of the opposite orientation whose extent matches `pos`, B
      - if `fixed` of A is inside the `extent` of B, record an intersection point.


In [1]:
from __future__ import annotations

import aocd

data = aocd.get_data(day=3, year=2019)

In [2]:
from dataclasses import dataclass
from enum import Enum
from typing import (
    Dict,
    Iterable,
    MutableSet,
    NamedTuple,
    Tuple,
    Type,
    TypeVar,
    overload,
)

T = TypeVar("T", bound="Segment")


class Pos(NamedTuple):
    """Position on the grid"""

    x: int
    y: int

    def for_dir(self, dir: Dir) -> int:
        """Get the fixed value for a given direction"""
        return getattr(self, dir.value)

    @property
    def distance(self):
        """Manhattan distance to the origin"""
        return sum(map(abs, self))

    @classmethod
    def from_fixed_var(cls, fixed: int, var: int, dir: Dir) -> Pos:
        """Create a new position given a fixed and variable value, and direction"""
        v = cls(var, fixed)
        return cls(v.for_dir(dir), v.for_dir(~dir))


class Dir(Enum):
    """Directions on the grid"""

    # value is the position attribute that is fixed
    v = "x"
    h = "y"

    @classmethod
    def from_direction(cls, direction: str) -> Dir:
        """Direction for a given string UDLR"""
        return cls.v if direction in "UD" else cls.h

    def __invert__(self) -> Dir:
        return Dir.v if self is Dir.h else Dir.h


@dataclass(frozen=True)
class Segment:
    dir: Dir
    fixed: int
    extent: range

    @classmethod
    def from_direction(
        cls, from_: Pos, direction: str, length: int
    ) -> Tuple[Segment, Pos]:
        """Given a position, direction and length, returns a segment and new position"""
        dir = Dir.from_direction(direction)
        start, fixed = map(from_.for_dir, (dir, ~dir))
        length += 1  # ranges are exclusive, we want inclusive
        start, end = start, start + length
        newpos = Pos.from_fixed_var(fixed, end - 1, dir)
        if direction in "DL":  # reverse direction
            start, end = (start - length + 1, end - length + 1)
            newpos = Pos.from_fixed_var(fixed, start, dir)
        return cls(dir, fixed, range(start, end)), newpos

    def intersections(self, line_map: LineMap[T]) -> Iterable[T]:
        """Given a line map, yield all intersections"""
        opposite_segments = line_map.get(~self.dir, {})
        for pos in self.extent:
            for opposite in opposite_segments.get(pos, ()):
                if self.fixed in opposite.extent:
                    yield opposite


LineMap = Dict[Dir, Dict[int, MutableSet[T]]]


@overload
def segments_from_directions(directions: Iterable[str]) -> Iterable[Segment]:
    ...


@overload
def segments_from_directions(directions: Iterable[str], S: Type[T]) -> Iterable[T]:
    ...


def segments_from_directions(
    directions: Iterable[str], S: Type = Segment
) -> Iterable[Segment]:
    pos = Pos(0, 0)
    for dir_length in directions:
        seg, pos = S.from_direction(pos, dir_length[0], int(dir_length[1:]))
        yield seg


def create_line_map(segments: Iterable[T]) -> LineMap[T]:
    map: LineMap[T] = {}
    for seg in segments:
        map.setdefault(seg.dir, {}).setdefault(seg.fixed, set()).add(seg)
    return map


def intersection_distances(
    line_map: LineMap[T], segments: Iterable[T]
) -> Iterable[int]:
    for seg in segments:
        for other in seg.intersections(line_map):
            distance = Pos.from_fixed_var(seg.fixed, other.fixed, seg.dir).distance
            if distance:
                yield distance


def min_distance(lines: str) -> int:
    line_a, line_b = (line.split(",") for line in lines.splitlines())
    line_map: LineMap[Segment] = create_line_map(segments_from_directions(line_a))
    distances = intersection_distances(line_map, segments_from_directions(line_b))
    return min(distances)


tests = {
    "R8,U5,L5,D3\nU7,R6,D4,L4": 6,
    ("R75,D30,R83,U83,L12,D49,R71,U7,L72\n" "U62,R66,U55,R34,D71,R55,D58,R83"): 159,
    (
        "R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51\n"
        "U98,R91,D20,R16,D67,R40,U7,R15,U6,R7"
    ): 135,
}
for lines, expected in tests.items():
    result = min_distance(lines)
    assert result == expected

In [3]:
print("Part 1:", min_distance(data))

Part 1: 2180


## Part 2

We have to track _steps_ now, instead of Manhattan distances. Note that a wire crossing itself affects this:

> If a wire visits a position on the grid multiple times, use the steps value from the first time it visits that position

This means that when processing the segments, we can record step information but have to record the lower step count for our own line at crossing points.

I at first thought that this meant we'd propagate that value throughout the wire (if, after 100 steps you cross your own wire that got there after 42 steps, the next step value is 43), but that resulted in too low an answer.


In [4]:
from dataclasses import field


@dataclass(frozen=True)
class StepSegment(Segment):
    # step count at different lengths along the segment
    steps: Dict[int, int] = field(default_factory=dict, hash=False)

    def stepcount(self, pos: int) -> int:
        return self.steps[pos]

    def update_stepcount(self, start: int, line_map: LineMap[StepSegment]) -> None:
        positions = (
            iter(self.extent) if start == self.extent[0] else reversed(self.extent)
        )
        # find all wires that visited this point before, so we can copy their step count.
        # Note that we always visited our starting point before, except for the first line.
        intersections = {s.fixed: s for s in self.intersections(line_map)}
        firstpos = next(positions)
        prev = intersections.get(firstpos)
        self.steps[firstpos] = step = 0 if not prev else prev.stepcount(self.fixed)
        for steps, pos in enumerate(positions, start=step + 1):
            other = intersections.get(pos)
            if other:
                # we visited this point before, use that count here
                steps = other.stepcount(self.fixed)
            self.steps[pos] = steps


def intersection_stepcounts(
    line_map: LineMap[StepSegment], segments: Iterable[StepSegment]
) -> Iterable[int]:
    for seg in segments:
        for other in seg.intersections(line_map):
            if not (seg.fixed and other.fixed):
                continue  # position 0, 0
            yield seg.stepcount(other.fixed) + other.stepcount(seg.fixed)


def propagate_steps(segments: Iterable[StepSegment]) -> Iterable[StepSegment]:
    map: LineMap[StepSegment] = {}
    last_pos = Pos(0, 0)
    for seg in segments:
        seg.update_stepcount(last_pos.for_dir(seg.dir), map)
        # determine what end to take, adjusting for L vs R, U vs D
        new_pos = Pos.from_fixed_var(seg.fixed, seg.extent[-1], seg.dir)
        if new_pos == last_pos:
            new_pos = Pos.from_fixed_var(seg.fixed, seg.extent[0], seg.dir)
        map.setdefault(seg.dir, {}).setdefault(seg.fixed, set()).add(seg)
        yield seg
        last_pos = new_pos


def min_steps(lines: str) -> int:
    split = (line.split(",") for line in lines.splitlines())
    a_segments, b_segments = (
        propagate_steps(segments_from_directions(line, S=StepSegment)) for line in split
    )
    line_map: LineMap[StepSegment] = create_line_map(a_segments)
    steps = intersection_stepcounts(line_map, b_segments)
    return min(steps)


tests = {
    "R8,U5,L5,D3\nU7,R6,D4,L4": 30,
    ("R75,D30,R83,U83,L12,D49,R71,U7,L72\n" "U62,R66,U55,R34,D71,R55,D58,R83"): 610,
    (
        "R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51\n"
        "U98,R91,D20,R16,D67,R40,U7,R15,U6,R7"
    ): 410,
}
for lines, expected in tests.items():
    result = min_steps(lines)
    assert result == expected

In [5]:
print("Part 2:", min_steps(data))

Part 2: 112316
